<a href="https://colab.research.google.com/github/JoeyAviles89z/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/Joey_Aviles_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

In [1]:
### YOUR CODE STARTS HERE
# Let's do it!
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data --no-check-certificate

--2020-04-09 03:12:58--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
  Issued certificate has expired.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.07s   

2020-04-09 03:12:59 (271 KB/s) - ‘house-votes-84.data’ saved [18171/18171]



In [2]:
import pandas as pd
df = pd.read_csv('house-votes-84.data')

print(df.shape)
df.head()

(434, 17)


,republican,n,y,n.1,y.1,y.2,y.3,n.2,n.3,n.4,y.4,?,y.5,y.6,y.7,n.5,y.8
0,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
1,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
2,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
3,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
4,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y


In [3]:
df = pd.read_csv('house-votes-84.data', names=['party', 'handicapped-infants', 'water-project', 'budget', 'physician-fee-freeze', 'el-salvador-aid', 'religious-groups', 'anti-satellite-ban', 'aid-to-contras', 'mx-missile', 'immigration', 'synfuels', 'education', 'right-to-sue', 'crime', 'duty-free', 'south-africa'])
print(df.shape)
df.head()

(435, 17)


,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [4]:
#how many from each party?
df['party'].value_counts()

democrat      267
republican    168
Name: party, dtype: int64

In [5]:
#Label encoding, and replace ? with Np.NaN in one fell-swoop.
import numpy as np

df = df.replace({'y': 1, 'n': 0, '?': np.NaN})

df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [0]:
# Split the dataset into republicans and democrats

rep = df[df['party'] == 'republican']
dem = df[df['party'] == 'democrat']

In [7]:
#how did Republicans vote?
rep.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [8]:
#how did Democrats vote?
dem.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


##Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01##

In [0]:
from scipy.stats import ttest_ind

In [10]:
#What is the mean support of Republicans?
rep['budget'].mean()

0.13414634146341464

In [11]:
#What is the mean support of Democrats?
dem['budget'].mean()

0.8884615384615384

After finding the mean of both democrats and republicans:

Null Hypothesis: There is no significant difference between republicans and democrats support for the budget.

Alternative Hypothesis: Democrats support the budget more than republicans.

In [12]:
#compare with a t-test:
ttest_ind(rep['budget'], dem['budget'])

Ttest_indResult(statistic=nan, pvalue=nan)

In [13]:
#account for NaN's
ttest_ind(rep['budget'], dem['budget'], nan_policy='omit')

Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77)

##Since p is lower than .01 (99%) Democrats hold a larger support for the budget than Republicans. In conclusion, I reject the null Hypothesis.

In [14]:
#You could also remove NaN values from this column

col = rep['budget']
rep_budget_no_nans = col[~np.isnan(col)]

col = dem['budget']
dem_budget_no_nans = col[~np.isnan(col)]

#My sample sizes for the two samples:
print(len(rep_budget_no_nans))
print(len(dem_budget_no_nans))

164
260


##Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01##

In [15]:
#the percentage of democrats who voted 'yes' on el-salvador-aid bill

dem['el-salvador-aid'].sum()/len(dem)

0.20599250936329588

In [16]:
#Remove NaN values from this column

col = dem['el-salvador-aid']

np.isnan(col)

el_salvador_aid_no_nans = col[~np.isnan(col)]

el_salvador_aid_no_nans

el_salvador_aid_no_nans.sum()/len(el_salvador_aid_no_nans)

0.21568627450980393

In [17]:
#the percentage of republicans who voted 'yes' on el-salvador-aid bill

rep['el-salvador-aid'].sum()/len(rep)

0.9345238095238095

In [18]:
#Remove NaN values from this column

col = rep['el-salvador-aid']

np.isnan(col)

el_salvador_aid_no_nans = col[~np.isnan(col)]

el_salvador_aid_no_nans

el_salvador_aid_no_nans.sum()/len(el_salvador_aid_no_nans)

0.9515151515151515

After finding the mean of both democrats and republicans:

Null Hypothesis: There is no significant difference between republicans and democrats support for el salvador aid.

Alternative Hypothesis: Republicans support for aid for el salvador is more than democrats.

In [19]:
#account for NaN's
ttest_ind(rep['el-salvador-aid'], dem['el-salvador-aid'], nan_policy='omit')

Ttest_indResult(statistic=21.13669261173219, pvalue=5.600520111729011e-68)

##Since p is lower than .01 (99%) Republicans hold a larger support for the Aid to El Salvador than Democrats. In conclusion, I reject the null Hypothesis.

##Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)##

In [20]:
#What is the mean support of Republicans?
rep['south-africa'].mean()

0.6575342465753424

In [21]:
#What is the mean support of Democrats?
dem['south-africa'].mean()

0.9351351351351351

After finding the mean of both democrats and republicans:

Null Hypothesis: There is a significant difference between republicans and democrats support for south africa.

Alternative Hypothesis: Republican and Democrat support for south africa is not that different.

In [22]:
#account for NaN's
ttest_ind(rep['south-africa'], dem['south-africa'], nan_policy='omit')

Ttest_indResult(statistic=-6.849454815841208, pvalue=3.652674361672226e-11)

###Since p is lower than .01 (99%) Republicans and Democrats support for south-africa is not that different. In conclusion, I reject the Null Hypothesis.

However, an instance in which p > .01 could be the water-project that was done in class yesterday.

In [0]:
from scipy.stats import ttest_ind

In [24]:
# What is the mean support of Republicans?
rep['water-project'].mean()

0.5067567567567568

In [25]:
# what is the mean support of Democrats?
dem['water-project'].mean()

0.502092050209205

In [26]:
# compare with a t-test:
ttest_ind(rep['water-project'], dem['water-project'])

Ttest_indResult(statistic=nan, pvalue=nan)

In [27]:
# account for NaN's
ttest_ind(rep['water-project'], dem['water-project'], nan_policy='omit')

Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)

In [28]:
# You could also remove NaN values from this column

col = rep['water-project']
rep_water_project_no_nans = col[~np.isnan(col)]

col = dem['water-project']
dem_water_project_no_nans = col[~np.isnan(col)]

# My sample sizes for the two samples:
print(len(rep_water_project_no_nans))
print(len(dem_water_project_no_nans))

148
239


4) T-statistic: .089

5) P-value: .929

I want to reject the null hypothesis if my p-value is < .01 or if my p-value is less than (1-confidence_level)

Conclusion: due to a p-value of .929 I fail to reject the null hypothesis that republican and democrat support for the water-project bill is different.

I never say that I "accept" the null hypothesis, I just say that I "fail to reject"

## Stretch Goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Work on Performing a T-test without using Scipy in order to get "under the hood" and learn more thoroughly about this topic.
### Start with a 1-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://blog.minitab.com/hs-fs/hubfs/Imported_Blog_Media/701f9c0efa98a38fb397f3c3ec459b66.png?width=247&height=172&name=701f9c0efa98a38fb397f3c3ec459b66.png) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then try a 2-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://lh3.googleusercontent.com/proxy/rJJ5ZOL9ZDvKOOeBihXoZDgfk7uv1YsRzSQ1Tc10RX-r2HrRpRLVqlE9CWX23csYQXcTniFwlBg3H-qR8MKJPBGnjwndqlhDX3JxoDE5Yg) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then check your Answers using Scipy!